# Swap
Agent $\alpha$ swaps asset $i$ with the protocol for asset $j$.

## Sell
Agent $\alpha$ sells asset $i$ in quantity $\Delta R_i$ for asset $j$.

### Constraints

The swap will keep the swap invariant for assets $i$ and $j$, $R_iQ_i$ and $R_jQ_j$, constant, in the case that fees are set to $0$.

## Updating AMM state

### Calculating swaps

#### Case 1: Asset $j$ bought in quantity $-\Delta R_j$ with asset $i$

$$
\begin{align}
\tilde{\Delta Q_j} &= Q_j\frac{- \Delta R_j}{R_j(1 - f_A) + \Delta R_j}\\
\tilde{\Delta Q_i} &= -\frac{\tilde{\Delta Q_j}}{1 - f_P}\\
\Delta R_i &= R_i\frac{- \tilde{\Delta Q_i}}{Q_i + \tilde{\Delta Q_i}}
\end{align}
$$

Now that the correct $\Delta R_i$ is computed, we can treat this trade as a "sell" trade of asset $i$ in quantity $\Delta R_i$.

#### Case 2: Asset $i$ sold in quantity $\Delta R_i$ for asset $j$

Asset $i$ is sold for asset $j$ in a series of trades, with 
$$
\begin{align}
\Delta Q_i &= Q_i\frac{- \Delta R_i}{R_i + \Delta R_i}\\
\Delta Q_j' &= -\Delta Q_i (1 - f_P)\\
\Delta R_j' &= R_j\frac{- \Delta Q_j'}{Q_j + \Delta Q_j'}\\
\Delta R_j &= \Delta R_j'(1 - f_A)
\end{align}
$$

### Fee accounting

The LRNA fee amount is $-\Delta Q_i f_P$, and it is burned until $L$ is brought up to $0$, at which point fees are moved to the LRNA side of the HDX sub-pool in Omnipool.
$$
\begin{align}
\Delta L &= min(-\Delta Q_i f_P, -L)\\
\Delta Q_H &= -\Delta Q_i f_P - \Delta L
\end{align}
$$
where $Q_H$ is the LRNA side of the HDX sub-pool.



## Direct calculation (updating pool state)
### $\Delta R_i > 0$ provided
$$
\begin{align}
\Delta Q_i &= Q_i \frac{-\Delta R_i}{R_i + \Delta R_i}\\
\Delta Q_j &= -\Delta Q_i(1 - f_P)\\
\Delta R_j &= R_j \frac{-\Delta Q_j}{Q_j + \Delta Q_j}(1 - f_A)\\
\Delta L &= min(-\Delta Q_i f_P, -L)\\
\Delta Q_H &= -\Delta Q_i f_P - \Delta L
\end{align}
$$

### $\Delta R_j < 0$ provided
$$
\begin{align}
\Delta Q_j &= Q_j\frac{-\Delta R_j}{R_j(1 - f_A) + \Delta R_j}\\
\Delta Q_i &= -\frac{\Delta Q_j}{1 - f_P}\\
\Delta R_i &= R_i\frac{-\Delta Q_i}{Q_i + \Delta Q_i}\\
\Delta L &= min(-\Delta Q_i f_P, -L)\\
\Delta Q_H &= -\Delta Q_i f_P - \Delta L
\end{align}
$$

## Updating agent state

We have
$$
\Delta r^\alpha_i = -\Delta R_i\\
\Delta r^\alpha_j = -\Delta R_j
$$

## Fee calculation

In other swap implementations, we calculate overall fee paid by the user in the asset not specified. Here we derive this for Omnipool. We calculate it as the difference between the amount out (in) when the amount in (out) is specified, between the existing fees and feeless case.

### $\Delta R_i > 0$ provided
We use a $0$ superscript to indicate feeless quantitie, i.e. the value of variables if both asset and protocol fees were zero.
$$
\begin{align}
fee &= \Delta R_j - \Delta R_j^0\\
&= R_j \frac{-\Delta Q_j}{Q_j + \Delta Q_j}(1 - f_A) - R_j \frac{-\Delta Q_j^0}{Q_j + \Delta Q_j^0}\\
&= R_j \frac{\Delta Q_i(1 - f_P)}{Q_j - \Delta Q_i(1 - f_P)}(1 - f_A) - R_j \frac{\Delta Q_i}{Q_j - \Delta Q_i}\\
&= R_j \left[\frac{\Delta Q_i(1 - f_P)(1 - f_A)(Q_j - \Delta Q_i)}{(Q_j - \Delta Q_i(1 - f_P))(Q_j - \Delta Q_i)} - \frac{\Delta Q_i(Q_j - \Delta Q_i(1 - f_P))}{(Q_j - \Delta Q_i)(Q_j - \Delta Q_i(1 - f_P))}\right]\\
&= R_j \left[\frac{\Delta Q_i(1 - f_P)(Q_j - \Delta Q_i - f_A Q_j + f_A \Delta Q_i) - \Delta Q_i(Q_j - \Delta Q_i(1 - f_P))}{(Q_j - \Delta Q_i)(Q_j - \Delta Q_i(1 - f_P))}\right]\\
&= R_j \left[\frac{\Delta Q_i(1 - f_P)(Q_j - f_A Q_j + f_A \Delta Q_i) - \Delta Q_i Q_j}{(Q_j - \Delta Q_i)(Q_j - \Delta Q_i(1 - f_P))}\right]\\
&= R_j \left[\frac{\Delta Q_i(-f_A Q_j + f_A \Delta Q_i - f_P(Q_j - f_A Q_j + f_A \Delta Q_i))}{(Q_j - \Delta Q_i)(Q_j - \Delta Q_i(1 - f_P))}\right]\\
&= R_j \left[\frac{\Delta Q_i(-f_A Q_j + f_A \Delta Q_i - f_PQ_j + f_P f_A Q_j -f_P f_A \Delta Q_i)}{(Q_j - \Delta Q_i)(Q_j - \Delta Q_i(1 - f_P))}\right]\\
&= R_j \left[\frac{\Delta Q_i(f_A (\Delta Q_i - Q_j) - f_PQ_j + f_P f_A (Q_j -\Delta Q_i))}{(Q_j - \Delta Q_i)(Q_j - \Delta Q_i(1 - f_P))}\right]\\
&= R_j \Delta Q_i\left[\frac{-f_A - \frac{f_PQ_j}{Q_j - \Delta Q_i} + f_P f_A}{Q_j - \Delta Q_i(1 - f_P)}\right]\\
&= R_j \Delta Q_i\left[\frac{- \frac{f_PQ_j}{Q_j - \Delta Q_i} - (1 - f_P) f_A}{Q_j - \Delta Q_i(1 - f_P)}\right]\\
&= R_j Q_i \frac{-\Delta R_i}{R_i + \Delta R_i}\left[\frac{- \frac{f_PQ_j}{Q_j - Q_i \frac{-\Delta R_i}{R_i + \Delta R_i}} - (1 - f_P) f_A}{Q_j - Q_i \frac{-\Delta R_i}{R_i + \Delta R_i}(1 - f_P)}\right]\\
&= R_j Q_i \frac{-\Delta R_i}{R_i + \Delta R_i}\left[\frac{- \frac{f_PQ_j(R_i + \Delta R_i)}{Q_j(R_i + \Delta R_i) + Q_i \Delta R_i} - (R_i + \Delta R_i)(1 - f_P) f_A}{Q_j(R_i + \Delta R_i) + Q_i \Delta R_i(1 - f_P)}\right]\\
&= -R_j Q_i \Delta R_i\left[\frac{- \frac{f_PQ_j}{Q_j(R_i + \Delta R_i) + Q_i \Delta R_i} - (1 - f_P) f_A}{Q_j(R_i + \Delta R_i) + Q_i \Delta R_i(1 - f_P)}\right]\\
&= R_j \frac{Q_i \Delta R_i}{Q_j(R_i + \Delta R_i) + Q_i \Delta R_i(1 - f_P)}\left[ \frac{f_PQ_j}{Q_j(R_i + \Delta R_i) + Q_i \Delta R_i} + (1 - f_P) f_A\right]\\
\end{align}
$$

## $\Delta R_j < 0$ provided

$$fee = \frac{Q_j R_i \Delta R_j}{\left(Q_i + \frac{Q_j \Delta R_j}{R_j + \Delta R_j}\right) \left(R_j + \Delta R_j\right)} - \frac{Q_j R_i \Delta R_j}{\left(1 - f_P\right) \left(Q_i + \frac{Q_j \Delta R_j}{\left(1 - f_P\right) \left(R_j \left(1 - f_A\right) + \Delta R_j\right)}\right) \left(R_j \left(1 - f_A\right) + \Delta R_j\right)}$$